In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
import os
import librosa
import librosa.display
import IPython.display as ipd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

2024-11-21 20:36:56.305363: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 20:36:56.311277: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-21 20:36:56.378212: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-21 20:36:56.435142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732196216.491543  104474 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732196216.50

In [2]:

def load_audio(AUDIO_PATH):
    audio, sr = librosa.load(AUDIO_PATH)
    return audio, sr

def wav2melSpec(AUDIO_PATH):
    audio, sr = librosa.load(AUDIO_PATH)
    return librosa.feature.melspectrogram(y=audio, sr=sr)

def imgSpec(ms_feature):
    fig, ax = plt.subplots()
    ms_dB = librosa.power_to_db(ms_feature, ref=np.max)
    print(ms_feature.shape)
    img = librosa.display.specshow(ms_dB, x_axis='time', y_axis='mel', ax=ax)
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set(title='Mel-frequency spectrogram');
    
def hear_audio(AUDIO_PATH):
    audio, sr = librosa.load(AUDIO_PATH)
    print("\t", end="")
    ipd.display(ipd.Audio(data=audio, rate=sr))
    
    
def get_audio_info(path, show_melspec=False, label=None):
    spec = wav2melSpec(path)
    if label is not None:
        print("Label:", label)
    if show_melspec is not False:
        imgSpec(spec)
    hear_audio(path)

In [3]:
for i in range(100):
    path = 'Snoring Dataset'
    fullpath = f'{path}/0/0_{i}.wav'
    spec = wav2melSpec(fullpath)
    print(spec.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'Snoring Dataset/0/0_0.wav'

In [4]:

MFCC_NUM =  14
SAMPLING_RATE = 44100 
MFCC_MAX_LEN = 128 
def wav2mfcc(wave, max_len=MFCC_MAX_LEN):
    mfcc = librosa.feature.mfcc(y=wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [5]:
snore_path =  "Snoring Dataset"

In [6]:
#By Yanapple https://www.kaggle.com/code/yanapple/speech-to-text-summer-project
import os
files_in_dir = os.listdir(snore_path)
mfcc_spectograms = []
i = 1

for e in range(1, 2001):
    file_name = f'1_109.wav' #Original f'{e}.wav' There is a space after the V1_1space(2)
    sampl = librosa.load(f'{snore_path}/1/{file_name}')[0]
    spectrogram = np.expand_dims(wav2mfcc(sampl), -1)
    mfcc_spectograms.append(spectrogram)
    sampl_shifted = librosa.effects.pitch_shift(sampl,sr = SAMPLING_RATE, n_steps=12)
    spectrogram_shifted = np.expand_dims(wav2mfcc(sampl_shifted), -1)
    mfcc_spectograms.append(spectrogram_shifted)

    if i % 100 == 0:
        print(i)
    i += 1

FileNotFoundError: [Errno 2] No such file or directory: 'Snoring Dataset'

In [24]:
import os
import numpy as np
import librosa

# Путь к директории с подкаталогами классов
base_path = "/home/egor/safeVision/code/Snoring Dataset"  # Убедитесь, что у вас правильный путь к директории

# Установим значение частоты дискретизации
SAMPLING_RATE = 44100  # Стандартная частота дискретизации для librosa

# # Функция для получения MFCC спектрограммы
# def wav2mfcc(audio, sr=SAMPLING_RATE, n_mfcc=14):
#     # Вычисление MFCC спектрограммы
#     mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
#     return mfcc

# Список для хранения MFCC спектрограмм
mfcc_spectograms = []

# Список классов (подкаталогов)
classes = ['0', '1']

# Обработка аудиофайлов в каждом классе
for class_label in classes:
    class_path = os.path.join(base_path, class_label)  # Путь к подкаталогу класса
    
    if os.path.exists(class_path):  # Проверяем, существует ли директория
        files_in_dir = os.listdir(class_path)  # Получаем список файлов в директории класса
        
        for file_name in files_in_dir:
            if file_name.endswith('.wav'):  # Обрабатываем только WAV файлы
                try:
                    # Загружаем аудиофайл
                    audio, _ = librosa.load(os.path.join(class_path, file_name), sr=SAMPLING_RATE)
                    
                    # Преобразуем в MFCC спектрограмму
                    spectrogram = np.expand_dims(wav2mfcc(audio), axis=-1)
                    
                    # Добавляем спектрограмму и метку
                    mfcc_spectograms.append((spectrogram, int(class_label)))  # Сохраняем кортеж (спектрограмма, метка)
                    
                    # Сдвиг тона и создание спектрограммы
                    audio_shifted = librosa.effects.pitch_shift(audio, sr=SAMPLING_RATE, n_steps=14)
                    spectrogram_shifted = np.expand_dims(wav2mfcc(audio_shifted), axis=-1)
                    
                    # Добавляем сдвинутую спектрограмму и метку
                    mfcc_spectograms.append((spectrogram_shifted, int(class_label)))  # Сохраняем кортеж (спектрограмма, метка)
                    
                    print(f"Processed file: {file_name} from class {class_label}")
                    
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")
    else:
        print(f"Class directory {class_path} does not exist.")

# Пример вывода результата
print(f"Total processed MFCC spectrograms: {len(mfcc_spectograms)}")

# Теперь разделим спектрограммы и метки
X = [x[0] for x in mfcc_spectograms]  # Спектрограммы
y = [x[1] for x in mfcc_spectograms]  # Метки классов

# Преобразуем в массивы NumPy
X = np.array(X)
y = np.array(y)

# Определяем границу для разделения на обучающие и тестовые данные
border = 1756  # Или другое значение, в зависимости от количества данных

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test = X[:border], X[border:]
y_train, y_test = y[:border], y[border:]

# Проверяем форму данных
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Processed file: 0_381.wav from class 0
Processed file: 0_490.wav from class 0
Processed file: 0_400.wav from class 0
Processed file: 0_478.wav from class 0
Processed file: 0_76.wav from class 0
Processed file: 0_311.wav from class 0
Processed file: 0_491.wav from class 0
Processed file: 0_14.wav from class 0
Processed file: 0_356.wav from class 0
Processed file: 0_329.wav from class 0
Processed file: 0_97.wav from class 0
Processed file: 0_287.wav from class 0
Processed file: 0_136.wav from class 0
Processed file: 0_394.wav from class 0
Processed file: 0_421.wav from class 0
Processed file: 0_140.wav from class 0
Processed file: 0_240.wav from class 0
Processed file: 0_448.wav from class 0
Processed file: 0_357.wav from class 0
Processed file: 0_459.wav from class 0
Processed file: 0_354.wav from class 0
Processed file: 0_104.wav from class 0
Processed file: 0_361.wav from class 0
Processed file: 0_229.wav from class 0
Processed file: 0_72.wav from class 0
Processed file: 0_495.wav fro

In [25]:
from tensorflow.keras import Sequential, layers

dropout = 0.15
input_dim = (14, 128, 1)  # Убедитесь, что это правильные размеры для ваших данных
output_dim_vocab = 1  # Мы изменили на 1 для двоичной классификации

model = Sequential([
    layers.Conv2D(filters=32, kernel_size=[6, 6], strides=[2, 2], input_shape=input_dim, padding="same", use_bias=False),
    layers.BatchNormalization(),
    layers.Activation(activation='leaky_relu'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Dropout(dropout),
    
    # 2 layer
    layers.Conv2D(128, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation(activation='relu'),
    layers.Dropout(dropout),
    
    # 3 conv layer
    layers.Conv2D(512, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation(activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(dropout),
    
    # 4 conv layer
    layers.Conv2D(512, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation(activation='relu'),
    layers.Dropout(dropout),
    
    layers.Flatten(),
    
    # Fully connected layer
    layers.Dense(256, activation="relu"),  # Изменено на relu
    layers.Dropout(dropout),
    
    # Output layer
    layers.Dense(1, activation="sigmoid"),  # Изменено на 1 для двоичной классификации
])

# Компиляция модели с двоичной кросс-энтропией
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'])


In [9]:

model_path = '/home/egor/safeVision/code/model'

earlystopper = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min')
checkpoint = ModelCheckpoint(f"{model_path}/model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tb_callback = TensorBoard(f'{model_path}/logs', update_freq=1)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, min_delta=1e-10, patience=3, verbose=1, mode='auto')

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 7, 64, 32)      │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 64, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 7, 64, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 2, 21, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2, 21, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2, 21, 128)     │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2, 21, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 2, 21, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2, 21, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 2, 21, 512)     │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 2, 21, 512)     │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 2, 21, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 10, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 10, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 10, 512)     │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1, 10, 512)     │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1, 10, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1, 10, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5120)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,310,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,304,257 (16.42 MB)

 Trainable params: 4,301,889 (16.41 MB)

 Non-trainable params: 2,368 (9.25 KB)

#Since I couldn't fix the snippet below. Even after trying different things. I'll stop Snoring.

In [26]:
#By Yanapple https://www.kaggle.com/code/yanapple/speech-to-text-summer-project

# Train the model
history = model.fit(
    x=X_train,
    y=y_train,#We don't have text train
    validation_split=0.1,
    epochs=50, #Original is 50
    batch_size=26,
    callbacks=[earlystopper, checkpoint, reduceLROnPlat, tb_callback]
)

Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8251 - loss: 0.8273 - precision: 0.6336 - recall: 0.7175
Epoch 1: val_loss did not improve from 0.01707
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.8268 - loss: 0.8186 - precision: 0.6371 - recall: 0.7203 - val_accuracy: 0.9737 - val_loss: 0.0907 - val_precision: 1.0000 - val_recall: 0.9737 - learning_rate: 0.0010
Epoch 2/50
51/53 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9801 - loss: 0.0618 - precision: 0.9722 - recall: 0.9434
Epoch 2: val_loss improved from 0.01707 to 0.00843, saving model to /home/egor/safeVision/code/model/model.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9803 - loss: 0.0615 - precision: 0.9721 - recall: 0.9447 - val_accuracy: 1.0000 - val_loss: 0.0084 - val_precision: 1.0000 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9833 - loss: 0.0393 - precision: 0.9747 - recall: 0.9604
Epoch 3: val_loss did not impr

In [27]:
model.save("/home/egor/safeVision/code/model/breath_2.keras")


In [28]:
# Оценка модели на тестовом наборе данных
test_loss, test_accuracy, precision, recall = model.evaluate(X_test, y_test, batch_size=26)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test precision: {precision}")
print(f"Test recall: {recall}")



2024-11-21 21:11:09.849970: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


ValueError: math domain error